In [1]:
import os
import logging
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

from tensorflow.keras.models import load_model
from tensorflow.keras.utils import get_custom_objects
import pandas as pd
import numpy as np
from tensorflow.keras.backend import sigmoid
import tensorflow as tf
import tensorflow.keras.backend as K
from multiprocessing.dummy import Pool as ThreadPool

def swish(x, beta = 1):
    # https://www.geeksforgeeks.org/ml-swish-function-by-google-in-keras/
    return (x * sigmoid(beta * x))
def leaky_relu(x):
    return tf.nn.leaky_relu(x, alpha=0.25)
def image_convert(a):
    new = np.zeros((30, 30))
    for i in range(30):
        if int(a[i]*100)==0:
            d = 29
        elif int(a[i]*100)==100:
            d = 0
        else:
            d = 30-int(30-(1-a[i])*30)
            if d==30:
                d = 29
        new[d, i] = 1
    return new

def df_parsing(data_all_, c_):
    data_predict = data_all_[[x for x in data_all_.columns.tolist() if c_ in x]]
    data_predict_ = data_predict.copy()
    data_predict_['日期'] = data_all_['日期']
    a = pd.melt(data_predict_, id_vars=['日期'], value_vars=[x for x in data_predict_.columns.tolist() if 'pr' not in x])
    b = pd.melt(data_predict_, id_vars=['日期'], value_vars=[x for x in data_predict_.columns.tolist() if 'pr' in x])
    a.columns = ['日期', 'var', 'before']
    b.columns = ['日期', 'var', 'after']
    b['var'] = b['var'].str.replace("-pr","")
    c = pd.merge(a,b, on = ["日期","var"] , how = "left")
    return c

get_custom_objects().update({'swish': swish})
get_custom_objects().update({'leaky_relu': leaky_relu})

In [2]:
banana_X_train = pd.read_pickle("banana_X_train.pkl")
banana_X_val = pd.read_pickle("banana_X_val.pkl")
banana_Y_train = pd.read_pickle("banana_Y_train.pkl")
banana_Y_val = pd.read_pickle("banana_Y_val.pkl")
cols_name = [[y+'-'+str(x).rjust(2,"0") for x in range(1,31)] for y in ['RSV','上價中位數','下價中位數','中價中位數','交易量','平均價','雨量']]
cols_name_2 = [x for x in banana_X_train.columns.tolist() if '-' not in x]

train_X_num = banana_X_train[cols_name_2[1:]].values
val_X_num = banana_X_val[cols_name_2[1:]].values

train_Y_1 = banana_Y_train[[x for x in banana_Y_train.columns if 'RSV' in x]].values
val_Y_1 = banana_Y_val[[x for x in banana_Y_val.columns if 'RSV' in x]].values

In [3]:
train_X_lstm_1 = []
for i in range(len(cols_name)):
    tmp = banana_X_train[cols_name[i]].iloc[:,-7:]
    train_X_lstm_1.append(tmp)
train_X_lstm_1 = np.stack(train_X_lstm_1,axis=2)
val_X_lstm_1 = []
for i in range(len(cols_name)):
    tmp = banana_X_val[cols_name[i]].iloc[:,-7:]
    val_X_lstm_1.append(tmp)
val_X_lstm_1 = np.stack(val_X_lstm_1,axis=2)

train_X_lstm_2 = []
for i in range(len(cols_name)):
    tmp_all = []
    tmp = banana_X_train[cols_name[i]].iloc[:,-13:]
    for j in range(7):
        tmp_1 = tmp.iloc[:,j:j+7]
        tmp_all.append(tmp_1)
    tmp_all = np.stack(tmp_all,axis=2)
    train_X_lstm_2.append(tmp_all)
train_X_lstm_2 = np.stack(train_X_lstm_2,axis=3)

val_X_lstm_2 = []
for i in range(len(cols_name)):
    tmp_all = []
    tmp = banana_X_val[cols_name[i]].iloc[:,-13:]
    for j in range(7):
        tmp_1 = tmp.iloc[:,j:j+7]
        tmp_all.append(tmp_1)
    tmp_all = np.stack(tmp_all,axis=2)
    val_X_lstm_2.append(tmp_all)
val_X_lstm_2 = np.stack(val_X_lstm_2,axis=3)

train_X_lstm_4 = np.stack([np.stack([image_convert(x) for x in banana_X_train[cols_name[y]].values]) for y in range(len(cols_name))], axis=3)
val_X_lstm_4 = np.stack([np.stack([image_convert(x) for x in banana_X_val[cols_name[y]].values]) for y in range(len(cols_name))], axis=3)

data_X = {'Conv1D':[train_X_lstm_1, val_X_lstm_1],
          'Vanilla':[train_X_lstm_1, val_X_lstm_1],
          'Stacked':[train_X_lstm_1, val_X_lstm_1],
          'Bidirectional':[train_X_lstm_1, val_X_lstm_1],
          'Conv1D_LSTM':[train_X_lstm_2, val_X_lstm_2],
          'Conv2D_1':[train_X_lstm_2, val_X_lstm_2],
          'Conv2D_2':[train_X_lstm_4, val_X_lstm_4]}

In [4]:
def model_predict(list_route_, model_, x1, x2, Y, dataset):
    #{'高於成本價格': 0, '低於成本價格': 1, '上漲': 0, '持平':1, '下跌':2}
    data_ = model_.predict([x1,x2], 
                           verbose = 0)#, use_multiprocessing = True, workers = 4, 
                           #max_queue_size = 20, batch_size = 256)
    data_ = data_[:,:,0]
    data_ = pd.DataFrame(data_)
    data_new = data_.copy()
    data_.columns = ['RSV-02-pr','RSV-03-pr','RSV-04-pr','RSV-05-pr','RSV-06-pr']
    data_new[(data_new>1)] = 1
    data_new[(data_new<0)] = 0
    data_new[data_new<=0.2] = 2
    data_new[(data_new>=0.8)&(data_new<=1)] = 0
    data_new[(data_new>0.2)&(data_new<0.8)] = 1
    data_new.columns = ['漲跌狀態-02-pr','漲跌狀態-03-pr','漲跌狀態-04-pr','漲跌狀態-05-pr','漲跌狀態-06-pr']
    data_ = pd.concat([data_, data_new] , axis = 1)
    data_all = Y.copy()
    for c_ in ['成本價格-02','成本價格-03','成本價格-04','成本價格-05','成本價格-06']:
        del data_all[c_]
    data_all = data_all.reset_index(drop = True)
    data_all = pd.concat([data_all, data_] , axis = 1)
    #data_RSV = df_parsing(data_all, c_ = "RSV")
    data_class = df_parsing(data_all, c_ = "漲跌狀態")
    data_class['result'] = ''
    data_class.loc[data_class['before']==data_class['after'], 'result'] = True
    data_class.loc[data_class['before']!=data_class['after'], 'result'] = False
    data_class_groupby = data_class.groupby(['var','before','result']).size().rename('num').reset_index(drop = False)
    data_class_groupby = data_class_groupby.pivot(index='var', columns=['before','result'], values='num')
    data_class_groupby.columns = [str(int(x[0]))+"-"+str(x[1]) for x in data_class_groupby.columns]
    data_class_groupby = data_class_groupby.reset_index()
    
    miss_col = list(set(['0-True', '1-True', '2-True', '0-False', '1-False', '2-False']) - set(data_class_groupby.columns.tolist()))
    if len(miss_col)>0:
        for c_ in miss_col:
            data_class_groupby[c_] = 0

    for c_ in ['0','1','2']:
        data_class_groupby[c_+"-per"] = data_class_groupby[c_+'-True']/(data_class_groupby[c_+'-True']+data_class_groupby[c_+'-False'])
    data_class_groupby = data_class_groupby.fillna(0)
    data_class_groupby['total-per'] = (data_class_groupby['0-True']+data_class_groupby['1-True']+data_class_groupby['2-True']) \
        /(data_class_groupby['0-True']+data_class_groupby['1-True']+data_class_groupby['2-True']+ \
          data_class_groupby['0-False']+data_class_groupby['1-False']+data_class_groupby['2-False'])
    data_class_groupby['source'] = list_route_
    data_all['source'] = list_route_
    data_class_groupby['dataset'] = dataset
    data_all['dataset'] = dataset
    
    evaluate_ = model_.evaluate([x1, x2], 
                                data_all[[x for x in data_all.columns if ('pr' not in x)&('RSV' in x)]].values,
                                verbose = 0)#, use_multiprocessing = True, workers = 4, 
                                #max_queue_size = 20, batch_size = 256)
    
    return data_class_groupby, data_all, {'source':list_route_, 'dataset':dataset, 'evaluate':evaluate_[1]}

def predict_all(list_route, data_X_, dataset):
    if dataset == "train":
        X = data_X_[list_route.split("/")[1]][0]
        Y = train_X_num
        al = banana_Y_train
    else:
        X = data_X_[list_route.split("/")[1]][1]
        Y = val_X_num
        al = banana_Y_val

    data_class_groupby = []
    data_all = []
    evaluate = []
    for h5 in ["final.h5", 'weights_loss.hdf5', 'weights_mse.hdf5']:
        list_route_ = list_route.replace("history.png", h5)
        final_model_ = load_model(list_route_)
        data_class_groupby_, data_all_, evaluate_ = model_predict(list_route_, final_model_, x1 = X, x2 = Y, Y = al, dataset = dataset)
        data_class_groupby.append(data_class_groupby_)
        data_all.append(data_all_)
        evaluate.append(evaluate_)
        del list_route_, final_model_, data_class_groupby_, data_all_, evaluate_
    data_class_groupby = pd.concat(data_class_groupby).reset_index(drop = True)
    data_all = pd.concat(data_all).reset_index(drop = True)
    return data_class_groupby, data_all, evaluate

def cal_all_acc(list_route_):
    data_all = []
    data_groupby = []
    data_evaluate = []
    data_error = []
    for i in range(len(list_route_)):
        print(i, len(list_route_))
        try:
            dataset_train = predict_all(list_route_[i], data_X, 'train')
            dataset_val = predict_all(list_route_[i], data_X, 'val')
            data_class_groupby_ = pd.concat([dataset_train[0], dataset_val[0]]).reset_index(drop = True)
            data_all_ = pd.concat([dataset_train[1], dataset_val[1]]).reset_index(drop = True)
            data_evaluate.append(dataset_train[2])
            data_evaluate.append(dataset_val[2])

            data_all.append(data_all_)
            data_groupby.append(data_class_groupby_)
            
            del dataset_train, dataset_val, data_class_groupby_, data_all_
            
        except:
            print("error", list_route_[i])
            data_error.append(list_route_[i])
    
    data_all = pd.concat(data_all).reset_index(drop = True)
    data_groupby = pd.concat(data_groupby).reset_index(drop = True)
    return data_all, data_groupby, data_evaluate, data_error

In [5]:
list_route = [os.path.join(path, name) for path, subdirs, files in os.walk('model') for name in files]
list_route = [x for x in list_route if 'history.png' in x]
print(len(list_route))
data = cal_all_acc(list_route)
data[0].to_csv("inference_all_banana_regression.csv", index = False)
data[1].to_csv("inference_groupby_banana_regression.csv", index = False)
pd.DataFrame(sum(data[2],[])).to_csv("inference_mse_banana_regression.csv", index = False)

1923
0 1923
1 1923
2 1923
3 1923
4 1923
5 1923
6 1923
7 1923
8 1923
9 1923
10 1923
11 1923
12 1923
13 1923
14 1923
15 1923
16 1923
17 1923
18 1923
19 1923
20 1923
21 1923
22 1923
23 1923
24 1923
25 1923
26 1923
27 1923
28 1923
29 1923
30 1923
31 1923
32 1923
33 1923
34 1923
35 1923
36 1923
37 1923
38 1923
39 1923
40 1923
41 1923
42 1923
43 1923
44 1923
45 1923
46 1923
47 1923
48 1923
49 1923
50 1923
51 1923
52 1923
53 1923
54 1923
55 1923
56 1923
57 1923
58 1923
59 1923
60 1923
61 1923
62 1923
63 1923
64 1923
65 1923
66 1923
67 1923
68 1923
69 1923
70 1923
71 1923
72 1923
73 1923
74 1923
75 1923
76 1923
77 1923
78 1923
79 1923
80 1923
81 1923
82 1923
83 1923
84 1923
85 1923
86 1923
87 1923
88 1923
89 1923
90 1923
91 1923
92 1923
93 1923
94 1923
95 1923
96 1923
97 1923
98 1923
99 1923
100 1923
101 1923
102 1923
103 1923
104 1923
105 1923
106 1923
107 1923
108 1923
109 1923
110 1923
111 1923
112 1923
113 1923
114 1923
115 1923
116 1923
117 1923
118 1923
119 1923
120 1923
121 1923
122 192

error model/Conv1D/0.01-sgd-tanh-leaky_relu/history.png
516 1923
error model/Conv1D/0.001-adam-leaky_relu-swish/history.png
517 1923
error model/Conv1D/0-adam-gelu-swish/history.png
518 1923
error model/Conv1D/0.01-adam-swish-swish/history.png
519 1923
error model/Conv1D/0.1-sgd-tanh-gelu/history.png
520 1923
error model/Conv1D/0-nadam-gelu-relu/history.png
521 1923
error model/Conv1D/0.1-sgd-tanh-swish/history.png
522 1923
error model/Conv1D/0.0001-nadam-leaky_relu-swish/history.png
523 1923
error model/Conv1D/0.001-adam-tanh-leaky_relu/history.png
524 1923
error model/Conv1D/0.0001-adam-tanh-swish/history.png
525 1923
error model/Conv1D/0.0001-adam-gelu-leaky_relu/history.png
526 1923
error model/Conv1D/0.001-sgd-tanh-gelu/history.png
527 1923
error model/Conv1D/0.1-nadam-swish-swish/history.png
528 1923
error model/Conv1D/0.001-nadam-relu-gelu/history.png
529 1923
error model/Conv1D/0-nadam-gelu-leaky_relu/history.png
530 1923
error model/Conv1D/0.01-adam-leaky_relu-leaky_relu/histo

error model/Conv2D_2/0.01-nadam-swish-leaky_relu/history.png
646 1923
error model/Conv2D_2/0.01-nadam-leaky_relu-leaky_relu/history.png
647 1923
error model/Conv2D_2/0.01-sgd-swish-relu/history.png
648 1923
error model/Conv2D_2/0.0001-sgd-gelu-swish/history.png
649 1923
error model/Conv2D_2/0-nadam-tanh-relu/history.png
650 1923
error model/Conv2D_2/0-adam-relu-swish/history.png
651 1923
error model/Conv2D_2/0-nadam-swish-leaky_relu/history.png
652 1923
error model/Conv2D_2/0.001-adam-leaky_relu-relu/history.png
653 1923
error model/Conv2D_2/0-nadam-tanh-swish/history.png
654 1923
error model/Conv2D_2/0.1-sgd-relu-swish/history.png
655 1923
error model/Conv2D_2/0-adam-leaky_relu-swish/history.png
656 1923
error model/Conv2D_2/0.01-adam-relu-swish/history.png
657 1923
error model/Conv2D_2/0-nadam-leaky_relu-swish/history.png
658 1923
error model/Conv2D_2/0.0001-adam-swish-leaky_relu/history.png
659 1923
error model/Conv2D_2/0.01-sgd-leaky_relu-relu/history.png
660 1923
error model/Conv2

error model/Conv2D_2/0-nadam-swish-swish/history.png
771 1923
error model/Conv2D_2/0.001-sgd-swish-leaky_relu/history.png
772 1923
error model/Conv2D_2/0.0001-nadam-leaky_relu-leaky_relu/history.png
773 1923
error model/Conv2D_2/0.1-sgd-relu-relu/history.png
774 1923
error model/Conv2D_2/0.01-adam-tanh-relu/history.png
775 1923
error model/Conv2D_2/0.1-nadam-gelu-leaky_relu/history.png
776 1923
error model/Conv2D_2/0.01-sgd-leaky_relu-swish/history.png
777 1923
error model/Conv2D_2/0.1-adam-tanh-gelu/history.png
778 1923
error model/Conv2D_2/0-adam-leaky_relu-relu/history.png
779 1923
error model/Conv2D_2/0.1-nadam-tanh-relu/history.png
780 1923
error model/Conv2D_2/0-nadam-gelu-gelu/history.png
781 1923
error model/Conv2D_2/0.001-nadam-gelu-leaky_relu/history.png
782 1923
error model/Conv2D_2/0.1-sgd-tanh-relu/history.png
783 1923
error model/Conv2D_2/0-nadam-relu-leaky_relu/history.png
784 1923
error model/Conv2D_2/0.0001-nadam-relu-relu/history.png
785 1923
error model/Conv2D_2/0.01

error model/Conv2D_2/0.01-nadam-gelu-swish/history.png
899 1923
error model/Conv2D_2/0.01-adam-gelu-swish/history.png
900 1923
error model/Stacked/0.01-sgd-gelu-swish/history.png
901 1923
error model/Stacked/0.001-nadam-swish-leaky_relu/history.png
902 1923
error model/Stacked/0.0001-adam-swish-gelu/history.png
903 1923
error model/Stacked/0-adam-swish-swish/history.png
904 1923
error model/Stacked/0.0001-nadam-tanh-relu/history.png
905 1923
error model/Stacked/0.001-adam-swish-swish/history.png
906 1923
error model/Stacked/0.1-adam-gelu-leaky_relu/history.png
907 1923
error model/Stacked/0-sgd-tanh-relu/history.png
908 1923
error model/Stacked/0.0001-nadam-tanh-gelu/history.png
909 1923
error model/Stacked/0.1-nadam-gelu-gelu/history.png
910 1923
error model/Stacked/0-sgd-leaky_relu-relu/history.png
911 1923
error model/Stacked/0.001-sgd-swish-relu/history.png
912 1923
error model/Stacked/0.001-nadam-relu-leaky_relu/history.png
913 1923
error model/Stacked/0.0001-sgd-leaky_relu-relu/h

...

error model/Conv1D_LSTM/0-sgd-tanh-relu/history.png
1384 1923
error model/Conv1D_LSTM/0.0001-nadam-tanh-gelu/history.png
1385 1923
error model/Conv1D_LSTM/0.1-nadam-gelu-gelu/history.png
1386 1923
error model/Conv1D_LSTM/0-sgd-leaky_relu-relu/history.png
1387 1923
error model/Conv1D_LSTM/0.001-sgd-swish-relu/history.png
1388 1923
error model/Conv1D_LSTM/0.0001-sgd-leaky_relu-relu/history.png
1389 1923
error model/Conv1D_LSTM/0.0001-nadam-swish-leaky_relu/history.png
1390 1923
error model/Conv1D_LSTM/0.0001-sgd-gelu-leaky_relu/history.png
1391 1923
error model/Conv1D_LSTM/0.01-sgd-swish-gelu/history.png
1392 1923
error model/Conv1D_LSTM/0.01-nadam-tanh-gelu/history.png
1393 1923
error model/Conv1D_LSTM/0.1-sgd-gelu-leaky_relu/history.png
1394 1923
error model/Conv1D_LSTM/0.1-nadam-leaky_relu-relu/history.png
1395 1923
error model/Conv1D_LSTM/0.01-nadam-gelu-relu/history.png
1396 1923
error model/Conv1D_LSTM/0.01-nadam-swish-swish/history.png
1397 1923
error model/Conv1D_LSTM/0.001-sgd-l

error model/Conv1D_LSTM/0.1-nadam-tanh-swish/history.png
1502 1923
error model/Conv1D_LSTM/0.0001-adam-leaky_relu-leaky_relu/history.png
1503 1923
error model/Conv1D_LSTM/0.0001-nadam-swish-gelu/history.png
1504 1923
error model/Conv1D_LSTM/0.1-adam-swish-leaky_relu/history.png
1505 1923
error model/Conv1D_LSTM/0.0001-nadam-swish-relu/history.png
1506 1923
error model/Conv1D_LSTM/0.01-adam-gelu-gelu/history.png
1507 1923
error model/Conv1D_LSTM/0.001-nadam-leaky_relu-gelu/history.png
1508 1923
error model/Conv1D_LSTM/0.001-sgd-gelu-leaky_relu/history.png
1509 1923
error model/Conv1D_LSTM/0.01-adam-tanh-swish/history.png
1510 1923
error model/Conv1D_LSTM/0-sgd-tanh-swish/history.png
1511 1923
error model/Conv1D_LSTM/0.001-nadam-relu-swish/history.png
1512 1923
error model/Conv1D_LSTM/0.001-sgd-swish-gelu/history.png
1513 1923
error model/Conv1D_LSTM/0.01-adam-gelu-leaky_relu/history.png
1514 1923
error model/Conv1D_LSTM/0.1-sgd-leaky_relu-swish/history.png
1515 1923
error model/Conv1D_L

error model/Conv1D_LSTM/0.01-nadam-gelu-swish/history.png
1622 1923
error model/Conv1D_LSTM/0.01-adam-gelu-swish/history.png
1623 1923
error model/Conv2D_1/0.01-sgd-gelu-swish/history.png
1624 1923
error model/Conv2D_1/0.001-nadam-swish-leaky_relu/history.png
1625 1923
error model/Conv2D_1/0.0001-adam-swish-gelu/history.png
1626 1923
error model/Conv2D_1/0-adam-swish-swish/history.png
1627 1923
error model/Conv2D_1/0.01-adam-relu-relu/history.png
1628 1923
error model/Conv2D_1/0.0001-nadam-tanh-relu/history.png
1629 1923
error model/Conv2D_1/0.001-adam-swish-swish/history.png
1630 1923
error model/Conv2D_1/0.1-adam-relu-relu/history.png
1631 1923
error model/Conv2D_1/0.1-adam-gelu-leaky_relu/history.png
1632 1923
error model/Conv2D_1/0-sgd-tanh-relu/history.png
1633 1923
error model/Conv2D_1/0.0001-nadam-tanh-gelu/history.png
1634 1923
error model/Conv2D_1/0.1-nadam-gelu-gelu/history.png
1635 1923
error model/Conv2D_1/0-sgd-leaky_relu-relu/history.png
1636 1923
error model/Conv2D_1/0.0

error model/Conv2D_1/0.001-sgd-leaky_relu-relu/history.png
1746 1923
error model/Conv2D_1/0.1-adam-leaky_relu-gelu/history.png
1747 1923
error model/Conv2D_1/0.0001-adam-tanh-leaky_relu/history.png
1748 1923
error model/Conv2D_1/0.01-nadam-tanh-leaky_relu/history.png
1749 1923
error model/Conv2D_1/0.1-adam-leaky_relu-relu/history.png
1750 1923
error model/Conv2D_1/0.0001-sgd-tanh-leaky_relu/history.png
1751 1923
error model/Conv2D_1/0.1-nadam-gelu-relu/history.png
1752 1923
error model/Conv2D_1/0.1-adam-swish-relu/history.png
1753 1923
error model/Conv2D_1/0.0001-nadam-tanh-leaky_relu/history.png
1754 1923
error model/Conv2D_1/0-adam-tanh-leaky_relu/history.png
1755 1923
error model/Conv2D_1/0.1-nadam-swish-gelu/history.png
1756 1923
error model/Conv2D_1/0.1-nadam-tanh-gelu/history.png
1757 1923
error model/Conv2D_1/0.1-sgd-swish-gelu/history.png
1758 1923
error model/Conv2D_1/0.001-adam-tanh-swish/history.png
1759 1923
error model/Conv2D_1/0.0001-nadam-relu-swish/history.png
1760 1923

error model/Conv2D_1/0-nadam-tanh-gelu/history.png
1870 1923
error model/Conv2D_1/0-adam-gelu-gelu/history.png
1871 1923
error model/Conv2D_1/0-nadam-leaky_relu-relu/history.png
1872 1923
error model/Conv2D_1/0-sgd-swish-swish/history.png
1873 1923
error model/Conv2D_1/0-nadam-relu-relu/history.png
1874 1923
error model/Conv2D_1/0.0001-sgd-tanh-gelu/history.png
1875 1923
error model/Conv2D_1/0.01-nadam-swish-relu/history.png
1876 1923
error model/Conv2D_1/0-sgd-leaky_relu-swish/history.png
1877 1923
error model/Conv2D_1/0-sgd-leaky_relu-gelu/history.png
1878 1923
error model/Conv2D_1/0-adam-gelu-relu/history.png
1879 1923
error model/Conv2D_1/0.1-adam-tanh-swish/history.png
1880 1923
error model/Conv2D_1/0.1-nadam-tanh-leaky_relu/history.png
1881 1923
error model/Conv2D_1/0.01-nadam-gelu-gelu/history.png
1882 1923
error model/Conv2D_1/0.01-adam-leaky_relu-swish/history.png
1883 1923
error model/Conv2D_1/0-nadam-swish-gelu/history.png
1884 1923
error model/Conv2D_1/0.001-adam-relu-relu/

In [6]:
exist_list = pd.concat([pd.read_csv("inference_groupby_banana_regression.csv"),
                        pd.read_csv("inference_groupby_banana_regression_2.csv"),
                        pd.read_csv("inference_groupby_banana_regression_3.csv"),
                        pd.read_csv("inference_groupby_banana_regression_4.csv"),
                        pd.read_csv("inference_groupby_banana_regression_5.csv"),
                        pd.read_csv("inference_groupby_banana_regression_6.csv"),
                        pd.read_csv("inference_groupby_banana_regression_7.csv"),
                       ]).reset_index(drop = True)
list_route = [os.path.join(path, name) for path, subdirs, files in os.walk('model') for name in files]
list_route = [x for x in list_route if 'history.png' in x]
list_route = list(set(list_route)-set(['/'.join(x.split("/")[:-1])+'/history.png' for x in pd.unique(exist_list['source'])]))
print(len(list_route))
data = cal_all_acc(list_route)
data[0].to_csv("inference_all_banana_regression_8.csv", index = False)
data[1].to_csv("inference_groupby_banana_regression_8.csv", index = False)
pd.DataFrame(sum(data[2],[])).to_csv("inference_mse_banana_regression_8.csv", index = False)

62
0 62
1 62
2 62
3 62
4 62
5 62
6 62
7 62
8 62
9 62
10 62
11 62
12 62
13 62
14 62
15 62
16 62
17 62
18 62
19 62
20 62
21 62
22 62
23 62
24 62
25 62
26 62
27 62
28 62
29 62
30 62
31 62
32 62
33 62
34 62
35 62
36 62
37 62
38 62
39 62
40 62
41 62
42 62
43 62
44 62
45 62
46 62
47 62
48 62
49 62
50 62
51 62
52 62
53 62
54 62
55 62
56 62
57 62
58 62
59 62
60 62
61 62


6114


In [5]:
df = pd.concat([pd.read_csv("inference_groupby_banana_regression.csv"),
                pd.read_csv("inference_groupby_banana_regression_2.csv"),
                pd.read_csv("inference_groupby_banana_regression_3.csv"),
                pd.read_csv("inference_groupby_banana_regression_4.csv"),
                pd.read_csv("inference_groupby_banana_regression_5.csv"),
                pd.read_csv("inference_groupby_banana_regression_6.csv"),
                pd.read_csv("inference_groupby_banana_regression_7.csv"),
                pd.read_csv("inference_groupby_banana_regression_8.csv"),
                       ]).reset_index(drop = True)
df.to_csv("regression.csv", index = False)

mse = pd.concat([pd.read_csv("inference_mse_banana_regression.csv"),
                pd.read_csv("inference_mse_banana_regression_2.csv"),
                pd.read_csv("inference_mse_banana_regression_3.csv"),
                pd.read_csv("inference_mse_banana_regression_4.csv"),
                pd.read_csv("inference_mse_banana_regression_5.csv"),
                pd.read_csv("inference_mse_banana_regression_6.csv"),
                pd.read_csv("inference_mse_banana_regression_7.csv"),
                pd.read_csv("inference_mse_banana_regression_8.csv")
                       ]).reset_index(drop = True)
mse.to_csv("mse.csv", index = False)

In [8]:
df[df['total-per']>0.7].groupby('source').size().rename('num').sort_values()

source
model/Conv2D_2/0.0001-nadam-tanh-relu/final.h5                          1
model/Conv1D_LSTM/0.001-adam-tanh-relu/weights_loss.hdf5                1
model/Conv1D_LSTM/0.001-adam-tanh-relu/final.h5                         1
model/Bidirectional/0.0001-nadam-swish-leaky_relu/weights_loss.hdf5     1
model/Bidirectional/0.0001-nadam-swish-leaky_relu/weights_mse.hdf5      1
                                                                       ..
model/Conv1D_LSTM/0-adam-relu-relu/weights_mse.hdf5                     9
model/Conv1D_LSTM/0-adam-relu-relu/weights_loss.hdf5                    9
model/Conv1D_LSTM/0-nadam-swish-relu/weights_mse.hdf5                  10
model/Conv1D_LSTM/0-nadam-swish-relu/weights_loss.hdf5                 10
model/Conv1D_LSTM/0-nadam-swish-relu/final.h5                          10
Name: num, Length: 966, dtype: int64

In [38]:
df[(df['source']=='model/Conv1D_LSTM/0-nadam-swish-relu/final.h5')]
#{'高於成本價格': 0, '低於成本價格': 1, '上漲': 0, '持平':1, '下跌':2}

,var,0-False,1-True,2-False,1-False,2-True,0-True,0-per,1-per,2-per,total-per,source,dataset
37740,漲跌狀態-02,173.0,979.0,162.0,463.0,888.0,1137.0,0.867939,0.678918,0.845714,0.790110,model/Conv1D_LSTM/0-nadam-swish-relu/final.h5,train
37741,漲跌狀態-03,141.0,1022.0,124.0,424.0,926.0,1165.0,0.892037,0.706777,0.881905,0.818780,model/Conv1D_LSTM/0-nadam-swish-relu/final.h5,train
37742,漲跌狀態-04,127.0,1023.0,121.0,422.0,919.0,1190.0,0.903569,0.707958,0.883654,0.823777,model/Conv1D_LSTM/0-nadam-swish-relu/final.h5,train
37743,漲跌狀態-05,125.0,990.0,108.0,442.0,956.0,1181.0,0.904288,0.691341,0.898496,0.822462,model/Conv1D_LSTM/0-nadam-swish-relu/final.h5,train
37744,漲跌狀態-06,146.0,958.0,114.0,486.0,937.0,1161.0,0.888294,0.663435,0.891532,0.803787,model/Conv1D_LSTM/0-nadam-swish-relu/final.h5,train
37755,漲跌狀態-02,49.0,132.0,48.0,104.0,167.0,172.0,0.778281,0.559322,0.776744,0.700893,model/Conv1D_LSTM/0-nadam-swish-relu/final.h5,val
37756,漲跌狀態-03,47.0,143.0,50.0,88.0,166.0,178.0,0.791111,0.619048,0.768519,0.724702,model/Conv1D_LSTM/0-nadam-swish-relu/final.h5,val
37757,漲跌狀態-04,39.0,140.0,44.0,91.0,183.0,175.0,0.817757,0.606061,0.806167,0.741071,model/Conv1D_LSTM/0-nadam-swish-relu/final.h5,val
37758,漲跌狀態-05,46.0,156.0,33.0,88.0,171.0,178.0,0.794643,0.639344,0.838235,0.751488,model/Conv1D_LSTM/0-nadam-swish-relu/final.h5,val
37759,漲跌狀態-06,49.0,131.0,50.0,101.0,167.0,174.0,0.780269,0.564655,0.769585,0.702381,model/Conv1D_LSTM/0-nadam-swish-relu/final.h5,val
